In [1]:
import requests
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

# 발급받은 인코딩된 서비스키를 사용하세요.
service_key = 'mUglWToAvQmeYwqfOR1UwESkdwoUdkYmZTS5lWDf5pEGuh1gvgyfIy%2BtFKzTNsFYqE%2BM0a6NlwJGtxyhBV63sQ%3D%3D'

def get_stock_code(stock_name):
    """종목명을 입력받아 종목 번호 조회"""
    url = f"https://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey={service_key}&numOfRows=1&pageNo=1&resultType=json&itmsNm={stock_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        items = data.get('response', {}).get('body', {}).get('items', {}).get('item', [])
        
        if items:
            # 종목 코드 추출
            stock_code = items[0].get('srtnCd', '')  # 짧은 종목 코드 가져오기
            market_type = items[0].get('mrktCtg', '')  # 시장 구분 (KOSPI/KOSDAQ)
            
            # yfinance 형식으로 변환
            suffix = '.KS' if market_type == 'KOSPI' else '.KQ'
            return f"{stock_code}{suffix}"
        else:
            print("해당 종목명을 찾을 수 없습니다.")
            return None
    else:
        print(f"API 호출 실패: {response.status_code}")
        return None

In [2]:
def get_stock_data(stock_name, period='5y', interval='monthly'):
    """종목명으로 yfinance 데이터를 불러오기"""
    ticker = get_stock_code(stock_name)
    if ticker is None:
        return None

    # yfinance로 주식 데이터 불러오기
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)

    # 원하는 데이터 재구성
#     if interval == 'weekly':  # 주 단위 데이터
#         df_resampled = df['Close'].resample('W-MON').last().reset_index()
    if interval == 'monthly':  # 월말 데이터
        df_resampled = df['Close'].resample('M').last().reset_index()
    elif interval == 'semi_monthly':  # 월중(15일) 및 월말 데이터
        # 날짜에서 필요한 정보 추출
        df = df.reset_index()  # 인덱스를 초기화하여 `Date` 열로 변환
        df['Day'] = df['Date'].dt.day
        df['Month'] = df['Date'].dt.month
        df['Year'] = df['Date'].dt.year

        # 월말 데이터: 각 달의 마지막 날짜
        df['Month_End'] = df['Date'] == df.groupby(['Year', 'Month'])['Date'].transform('max')
        
        # 월중(15일) 데이터와 월말 데이터 필터링
        mid_month_data = df[df['Day'] == 15]  # 월중 (15일)
        end_month_data = df[df['Month_End']]  # 월말 데이터
        
        # 월중 및 월말 데이터를 병합
        df_resampled = pd.concat([mid_month_data, end_month_data]).sort_values('Date').reset_index(drop=True)
    else:
        raise ValueError("지원되지 않는 interval 값입니다. 'weekly', 'monthly', 'semi_monthly' 중 하나를 선택하세요.")

    # 필요한 열만 반환
    return df_resampled[['Date', 'Close']]

def cal_increase(stock_name, interval):
    if interval == 'monthly':
        df_monthly = get_stock_data(stock_name, interval='monthly')
        df_monthly['Increase'] = df_monthly['Close'].diff()
        df_monthly = df_monthly.dropna().reset_index(drop=True)
        return df_monthly
    elif interval == 'semi_monthly':
        df_semi_monthly = get_stock_data(stock_name, interval='semi_monthly')
        df_semi_monthly['Increase'] = df_semi_monthly['Close'].diff()
        df_semi_monthly = df_semi_monthly.dropna().reset_index(drop=True)
        return df_semi_monthly

In [3]:
def predict_increase(stock_name, interval='monthly'):
    if interval == 'monthly':
        df_monthly = cal_increase(stock_name, interval='monthly')
        # SARIMA 모델 적용 (계절 주기 12 설정)
        model = SARIMAX(df_monthly['Increase'], order=(2, 1, 1), seasonal_order=(0, 1, 1, 12))
        monthly_model_fit = model.fit()

        # 12개월(1년) 예측
        monthly_forecast = monthly_model_fit.forecast(steps=12)

        # 예측 월 생성 (마지막 관측 월 이후 12개월)
        monthly_start_date = df_monthly['Date'].iloc[-1]  # 마지막 데이터의 날짜
        monthly_forecast_dates = pd.date_range(start=monthly_start_date + pd.DateOffset(months=1), periods=12, freq='M')

        # 예측 결과와 월 정보를 함께 데이터프레임으로 정리
        monthly_forecast_df = pd.DataFrame({'Date': monthly_forecast_dates, 'Predicted_Increase': monthly_forecast})
        return monthly_forecast_df
    elif interval =='semi_monthly':
        df_semi_monthly = cal_increase(stock_name, interval='semi_monthly')
        
        # 'Date' 열을 datetime 형식으로 변환
        if 'Date' in df_semi_monthly.columns and not pd.api.types.is_datetime64_any_dtype(df_semi_monthly['Date']):
            df_semi_monthly['Date'] = pd.to_datetime(df_semi_monthly['Date'])

        # 'Date'를 인덱스로 설정
        if df_semi_monthly.index.name != 'Date':
            df_semi_monthly.set_index('Date', inplace=True)

        model = SARIMAX(df_semi_monthly['Increase'], order=(2, 1, 1), seasonal_order=(0, 1, 1, 12))
        semi_monthly_model_fit = model.fit()

        # 12개월 예측
        semi_monthly_forecast = semi_monthly_model_fit.forecast(steps=24)

        # 예측 날짜 생성: 15일과 월말
        last_date = df_semi_monthly.index[-1]
        forecast_dates = []
        for i in range(1, 13):  # 12개월 동안
            mid_month = (last_date + pd.DateOffset(months=i)).replace(day=15)  # 15일
            end_month = (last_date + pd.DateOffset(months=i + 1)).replace(day=1) - pd.DateOffset(days=1)  # 월말
            forecast_dates.extend([mid_month, end_month])

        # 결과 데이터프레임 생성
        semi_monthly_forecast_df = pd.DataFrame({
            'Date': forecast_dates, 
            'Predicted_Increase': semi_monthly_forecast
        })
        return semi_monthly_forecast_df

In [4]:
monthly_pred = predict_increase("SK하이닉스", 'monthly')
monthly_pred

C:\Users\User\.conda\envs\test01\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


,Date,Predicted_Increase
60,2024-12-31 00:00:00+09:00,4219.942590
61,2025-01-31 00:00:00+09:00,3733.236002
62,2025-02-28 00:00:00+09:00,5822.308921
63,2025-03-31 00:00:00+09:00,3577.390719
64,2025-04-30 00:00:00+09:00,-5038.372365
65,2025-05-31 00:00:00+09:00,7035.807899
66,2025-06-30 00:00:00+09:00,13713.278399
67,2025-07-31 00:00:00+09:00,-15853.020532
68,2025-08-31 00:00:00+09:00,-11460.498447
69,2025-09-30 00:00:00+09:00,-4883.262529


In [5]:
semi_monthly_pred = predict_increase("SK하이닉스", 'semi_monthly')
semi_monthly_pred

C:\Users\User\.conda\envs\test01\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\User\.conda\envs\test01\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\User\.conda\envs\test01\lib\site-packages\statsmodels\tsa\base\tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


,Date,Predicted_Increase
100,2024-12-15 00:00:00+09:00,-187.080964
101,2024-12-31 00:00:00+09:00,-1062.906489
102,2025-01-15 00:00:00+09:00,6017.226033
103,2025-01-31 00:00:00+09:00,10722.726324
104,2025-02-15 00:00:00+09:00,-1325.826587
105,2025-02-28 00:00:00+09:00,-10832.121220
106,2025-03-15 00:00:00+09:00,-5411.396829
107,2025-03-31 00:00:00+09:00,2386.128792
108,2025-04-15 00:00:00+09:00,3996.250017
109,2025-04-30 00:00:00+09:00,-547.436479
